In [ ]:
import time
from pathlib import Path
import mimetypes
import whisper

In [ ]:
PATH = "/Users/gauthier/Documents/Travail/Projets/IFG/0030-2505 ZEPHYR/Période 3/05 Marketing opérationnel et digital/03 sharedownVideos"

In [ ]:
whisper.available_models()

In [ ]:
model = whisper.load_model("turbo")

def speech_to_text(path):
    result = model.transcribe(path)

    # Écriture dans le fichier txt

    txt_path = path + ".txt"

    with open(txt_path, "w", encoding="utf-8") as f:
        for segment in result['segments']:
            parole = segment["text"]
            f.write(parole + "\n")

In [ ]:
AUDIO_EXTS = {'.mp3', '.aac', '.m4a', '.ogg', '.oga', '.wav', '.flac', '.wma', '.aiff', '.alac', '.opus'}
VIDEO_EXTS = {'.mp4', '.m4v', '.mov', '.avi', '.mkv', '.webm', '.wmv', '.flv', '.mpeg', '.mpg', '.3gp', '.ts'}
MEDIA_EXTS = AUDIO_EXTS | VIDEO_EXTS

def _is_media_file(p: Path) -> bool:
    # 1) par extension (rapide)
    if p.suffix.lower() in MEDIA_EXTS:
        return True
    # 2) secours: type MIME (au cas où l’extension est atypique)
    mt, _ = mimetypes.guess_type(p.name)
    return isinstance(mt, str) and (mt.startswith('audio/') or mt.startswith('video/'))

def lister_medias(dossier: str | Path, recursive: bool = False, retourner_paths: bool = False) -> list[str] | list[Path]:
    """
    Retourne la liste des fichiers audio/vidéo contenus dans `dossier`.
    - recursive=False : uniquement le dossier courant
    - recursive=True  : parcourt tout l’arborescence
    - retourner_paths=False : retourne des str ; True : retourne des objets Path
    """
    d = Path(dossier).expanduser().resolve()
    if not d.is_dir():
        raise NotADirectoryError(f"'{d}' n'est pas un dossier existant.")

    it = d.rglob('*') if recursive else d.iterdir()
    fichiers = [p for p in it if p.is_file() and _is_media_file(p)]
    fichiers = sorted(fichiers, key=lambda p: p.name.lower())

    return fichiers if retourner_paths else [str(p) for p in fichiers]

In [ ]:
def transcript_folder(folder_path):

    medias = lister_medias(folder_path, recursive=True)

    len_media = len(medias)
    print(f"{len_media} fichier(s) compatible(s)\n")

    # chrono global
    t0 = time.time()

    for i, path in enumerate(medias, start=1):
        filename = Path(path).name
        print(f"{i} / {len_media}, traitement du fichier : {filename}")

        t1 = time.time()   # début du fichier
        try:
            speech_to_text(path)
            dt = time.time() - t1
            print(f"✅ La transcription de {filename} a eu lieu (temps : {dt:.1f} s).")
        except Exception as e:
            dt = time.time() - t1
            print(f"❌ {filename} n'a pas pu être transcrit (temps : {dt:.1f} s). Raison : {e}")

    # temps total
    total_time = time.time() - t0
    print("\n--- Résumé ---")
    print(f"Temps total : {total_time:.1f} s ({total_time/60:.1f} min)")

In [ ]:
transcript_folder(PATH.replace("\\", ""))